In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

newnames_reco = {"aice_body-sharp":"DLR", 
                 "first_body":"MBIR", 
                 "fbp_fc08":"FBP", 
                 "aidr3d_fc08":"HIR", 
                 "asir_standard":"HIR", 
                 "fbp_standard":"FBP",
                 "qr40f-q3_0.4":"Qr40f-Q3_0.4",
                 "qr40f-q3_0.6":"Qr40f-Q3_0.6",
                 "qr40f-q3_1":"Qr40f-Q3_1",
                 "br40f-q3_0.4":"Br40f-Q3_0.4",
                 "br40f-q3_0.6":"Br40f-Q3_0.6",
                 "br40f-q3_1":"Br40f-Q3_1"}

newnames_ct = {
    "CT4": "<b>Vendor 1",
    "GE": "<b>Vendor 2",
    "photon":"<b>Vendor 3",
}

# newnames_network = {
#     "3d_fullres_LiTS_151":"<b>3D",
#     "3d_fullres_LiTS_151_res":"<b>3Dres",
#     "3d_fullres_LiTS+phantom_250":"<b>3D improved",
#     "3d_fullres_LiTS+phantom_250_res":"<b>3Dres improved"
# }

newnames_network = {
    '3d_fullres_LiTS_151': 'LiTS',
    '3d_fullres_LiTS_151_res': 'LiTS',
    '3d_fullres_LiTS+phantom_250': 'LiTS+Phantom_10scans',
    '3d_fullres_LiTS+phantom_250_res': 'LiTS+Phantom_10scans',
    '3d_fullres_LiTS+phantom_257': 'LiTS+Phantom_20scans',
    '3d_fullres_LiTS+phantom_257_res': 'LiTS+Phantom_20scans',
    #'3d_fullres_LiTS+phantom_247': 'LiTS+Phantom_40scans',
    #'3d_fullres_LiTS+phantom_247_res': 'LiTS+Phantom_40scans',
    '3d_fullres_LiTS+phantom_254': 'LiTS+Phantom_40scans_10-200mA',
    '3d_fullres_LiTS+phantom_254_res': 'LiTS+Phantom_40scans_10-200mA',
}

network_types = {
    '3d_fullres_LiTS_151': '3d',
    '3d_fullres_LiTS_151_res': '3d_res',
    '3d_fullres_LiTS+phantom_250': '3d',
    '3d_fullres_LiTS+phantom_250_res': '3d_res',
    '3d_fullres_LiTS+phantom_257': '3d',
    '3d_fullres_LiTS+phantom_257_res': '3d_res',
    #'3d_fullres_LiTS+phantom_247': '3D orig',
    #'3d_fullres_LiTS+phantom_247_res': '3D residual',
    '3d_fullres_LiTS+phantom_254': '3d',
    '3d_fullres_LiTS+phantom_254_res': '3d_res',
}


dice_detection_threshold = 0
size_filter = 4/3 * 3.14159265359 * (5/2)**3   # in cubic millimeters, 5mm sphere, ignore all predictions smaller than this

output_dir = '../../plots/3) deep learning performance eval/network_improvement'


In [ ]:
df_lesion_meta = pd.read_csv('results_csv/results_lesion_paper.csv')

df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"volume_pred"] = None
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"dice"] = None
df_lesion_meta = df_lesion_meta[(df_lesion_meta["volume_pred"] > size_filter) | (df_lesion_meta["volume_pred"].isna()) ]

df_lesion_meta["detected"] = df_lesion_meta["dice"] > dice_detection_threshold
filter_lesion_props = (df_lesion_meta["C in HU"] < -10) | (df_lesion_meta["C in HU"].isna())
df_lesion_meta = df_lesion_meta[filter_lesion_props]
df_lesion_meta = df_lesion_meta[df_lesion_meta["kernel"] != "FC13"]

df_lesion_meta["ct"] = df_lesion_meta["ct"].map(newnames_ct)
df_lesion_meta = df_lesion_meta[df_lesion_meta["date"].isin([230915, 231205, 240306])]

In [ ]:
df_lesion_meta["lesion_prop"] = df_lesion_meta.apply(lambda x: f"{str(2*int(x['R in mm'])).zfill(2)}mm/{int(x['C in HU'])}HU" if not np.isnan(x['R in mm']) else None , axis=1)

df_lesion_meta["reco+kernel"] = df_lesion_meta["reco"].str.lower() + "_" + df_lesion_meta["kernel"].str.lower()
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco+kernel"].map(newnames_reco)
df_lesion_meta["network_type"] = df_lesion_meta["network"].map(network_types)
df_lesion_meta["network"] = df_lesion_meta["network"].map(newnames_network)


In [ ]:
groupby_lesion_prop = ["network", "network_type",  "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU"]
groupby_lesion_prop_rep = ["network", "network_type",  "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "rep"]
groupby_lesion_all = ["network", "network_type", "CTDIvol", "reco+kernel", "ct"]

In [ ]:
df_lesion_meta_recall = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 

df_lesion_meta_recall = df_lesion_meta_recall.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall.columns = groupby_lesion_all + ["recall", "num_detected", "num_total"]

df_lesion_meta["FP"] = df_lesion_meta["filename_gt"].isna() & df_lesion_meta["filename_pred"].notna() & (df_lesion_meta["px_numb_pred"] >= 0)

df_lesion_meta_precision = df_lesion_meta.copy()

df_lesion_meta_precision = df_lesion_meta_precision.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision.columns = groupby_lesion_all + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary = df_lesion_meta_recall.merge(df_lesion_meta_precision, on=groupby_lesion_all, how="left")
lesion_seg_summary["F1-score"] = 2 * (lesion_seg_summary["precision"] * lesion_seg_summary["recall"]) / (lesion_seg_summary["precision"] + lesion_seg_summary["recall"])
lesion_seg_summary["F1-score"] = lesion_seg_summary["F1-score"].fillna(0)

In [ ]:
df_lesion_meta_recall_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall_rep = df_lesion_meta_recall_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_rep.columns = groupby_lesion_all + ["rep"] + ["recall", "num_detected", "num_total"]

df_lesion_meta_precision_rep = df_lesion_meta.copy()
df_lesion_meta_precision_rep = df_lesion_meta_precision_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision_rep.columns = groupby_lesion_all + ["rep"] + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary_rep = df_lesion_meta_recall_rep.merge(df_lesion_meta_precision_rep, on=groupby_lesion_all + ["rep"], how="left")
lesion_seg_summary_rep["F1-score"] = 2 * (lesion_seg_summary_rep["precision"] * lesion_seg_summary_rep["recall"]) / (lesion_seg_summary_rep["precision"] + lesion_seg_summary_rep["recall"])
lesion_seg_summary_rep["F1-score"] = lesion_seg_summary_rep["F1-score"].fillna(0)



In [ ]:
lesion_seg_summary_rep_mean = lesion_seg_summary_rep.groupby(groupby_lesion_all).agg({"F1-score": "mean", "recall": "mean", "precision": "mean", "num_detected_x": "mean", "num_total_x": "mean", "dice_mean": "mean", "dice_std": "mean"}).reset_index()
lesion_seg_summary_rep_mean.columns = groupby_lesion_all + ["F1-score", "recall", "precision", "num_detected", "num_total", "dice_mean", "dice_std"]

In [ ]:
df_lesion_meta_recall_rep_lesion_prop = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall_rep_lesion_prop = df_lesion_meta_recall_rep_lesion_prop.groupby(groupby_lesion_prop_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_rep_lesion_prop.columns = groupby_lesion_prop_rep + ["recall", "num_detected", "num_total"]

df_lesion_meta_recall_rep_lesion_prop_mean = df_lesion_meta_recall_rep_lesion_prop.groupby(groupby_lesion_prop).agg({"recall": ["min", "mean", "std"], "num_detected": ["mean", "std"], "num_total": ["mean", "std"]}).reset_index()
df_lesion_meta_recall_rep_lesion_prop_mean.columns = groupby_lesion_prop + ["recall_min", "recall_mean", "recall_std", "num_detected_mean", "num_detected_std", "num_total_mean", "num_total_std"]


In [ ]:
lesion_seg_summary["CTDIvol"] = lesion_seg_summary["CTDIvol"].astype(float).round(5).astype(str)
lesion_seg_summary_rep["CTDIvol"] = lesion_seg_summary_rep["CTDIvol"].astype(float).round(5).astype(str)

In [ ]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1")]
lesion_seg_summary_rep_plot =  lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["network"].isin(["<b>3D improved", "<b>3Dres improved"])]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="reco+kernel", facet_row="ct", facet_col="network", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D improved", "<b>3Dres improved"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_improved_points_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
# calc AUC for each curve F1 vs CTDIvol
from sklearn.metrics import auc

lesion_seg_summary_rep_auc = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3Dres improved", "<b>3D improved"]))]
lesion_seg_summary_rep_auc = lesion_seg_summary_rep_auc[lesion_seg_summary_rep_auc["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]
aucs = []
for group, df in lesion_seg_summary_rep_auc.groupby(["ct", "network", "reco+kernel", "rep"]):
    x = df["CTDIvol"]
    x = x.astype(float)
    x = x/x.max()
    y = df["F1-score"]
    aucs.append((group, auc(x, y)))

aucs
aucs_df = pd.DataFrame(aucs, columns=["group", "AUC"])
# split group into network and reco+kernel and rep and keep AUC column
aucs_df["ct"] = aucs_df["group"].apply(lambda x: x[0])
aucs_df["network"] = aucs_df["group"].apply(lambda x: x[1])
aucs_df["reco+kernel"] = aucs_df["group"].apply(lambda x: x[2])
aucs_df["rep"] = aucs_df["group"].apply(lambda x: x[3])
aucs_df = aucs_df.drop(columns=["group"])

aucs_df_mean = aucs_df.groupby(["ct","network", "reco+kernel"]).agg({"AUC":["mean", "std"]}).reset_index()
aucs_df_mean.columns = ["ct", "network", "reco+kernel", "AUC_mean", "AUC_std"]


In [ ]:
from scipy.stats import wilcoxon, mannwhitneyu
for network in aucs_df["network"].unique():
    for ct in aucs_df["ct"].unique():
        for reco in aucs_df["reco+kernel"].unique():
            for reco_2 in aucs_df["reco+kernel"].unique()[1:]:
                if reco != reco_2:
                    aucs_reco = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco) & (aucs_df["ct"] == ct)]["AUC"]
                    aucs_reco_2 = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco_2) & (aucs_df["ct"] == ct)]["AUC"]
                    if len(aucs_reco) == 0 or len(aucs_reco_2) == 0:
                        continue    
                    t, p = mannwhitneyu(aucs_reco, aucs_reco_2)
                    print(f"{ct} {network} {reco} vs {reco_2} p-value: {p*12}")

In [ ]:
fig = px.bar(aucs_df_mean, x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="reco+kernel", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="ct", facet_col="network",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D improved", "<b>3Dres improved"]},
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=600
       )
fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_recos_dicetreshold_{dice_detection_threshold}.png"))
    


In [ ]:
fig = px.bar(aucs_df_mean[aucs_df_mean["reco+kernel"].isin(["HIR"])], x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="network", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="ct",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D improved", "<b>3Dres improved"]},
                        barmode="group",
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=330
       
       )
fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_networks_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
fig = px.bar(aucs_df_mean[aucs_df_mean["reco+kernel"].isin([ "HIR"]) ], x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="ct", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="network",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D improved", "<b>3Dres improved"]},
                        barmode="group",
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=300
       
       )
fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_vendors_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
lesion_seg_summary_comp = lesion_seg_summary_rep[(lesion_seg_summary_rep["reco+kernel"].isin(["HIR"]))]
lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["reco+kernel"].isin(["HIR", "1"]))]
lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["network"].isin(["<b>3D improved", "<b>3Dres improved"]))]
lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(float).round(2)
lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(str)
cdtivalues = lesion_seg_summary_comp["CTDIvol"].unique()
fig = px.scatter(lesion_seg_summary_comp,
              x="CTDIvol", y="F1-score", 
              color="ct", facet_row="network", 
              title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
              category_orders={"reco+kernel": ["FBP", "HIR",], "CTDIvol": cdtivalues, 
                               "network": ["<b>3D improved", "<b>3Dres improved"]})

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=700,
    width=550
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_vendors_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
lesion_seg_summary_comp = lesion_seg_summary_rep[(lesion_seg_summary_rep["reco+kernel"].isin(["HIR"]))]
lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["reco+kernel"].isin(["HIR", "1"]))]
lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["network"].isin(["<b>3D improved", "<b>3Dres improved"]))]
lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(float).round(2)
lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(str)
cdtivalues = lesion_seg_summary_comp["CTDIvol"].unique()
fig = px.scatter(lesion_seg_summary_comp,
              x="CTDIvol", y="F1-score", 
              color="network", facet_row="ct", 
              title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
              category_orders={"CTDIvol": cdtivalues,
                               "network": ["<b>3D improved", "<b>3Dres improved"]})

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=700,
    width=550
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_networks_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep["network"].isin(["<b>3Dres", "<b>3Dres improved"]))],
             x="CTDIvol", y="F1-score",
             color="reco+kernel", facet_row="ct", facet_col="network", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_recos_canon_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & 
                                      (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3D improved", "<b>3Dres", "<b>3Dres improved"])) &
                                      (lesion_seg_summary_rep["reco+kernel"] == "HIR")],
              x="CTDIvol", y="F1-score",
             color="network", facet_row="ct", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"network": ["<b>3D improved", "<b>3D", "<b>3Dres improved", "<b>3Dres"]}, facet_col="network_type")    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)


fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_points_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(newnames_network.values())) &
                       (lesion_seg_summary_rep["reco+kernel"] == "HIR")].to_csv(f"/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/promotion/daten_philipp/Figure 6/D/lesion_detection_network_improvement.csv")

In [ ]:

fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & 
                                      (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3D improved", 
                                                                               "3d_fullres_LiTS_151",
                                                                               "3d_fullres_LiTS+phantom_254",
                                                                               "3d_fullres_LiTS+phantom_255",
                                                                               "3d_fullres_LiTS+phantom_256",
                                                                               "3d_fullres_LiTS+phantom_246",
                                                                               "3d_fullres_LiTS+phantom_247",
                                                                               "3d_fullres_LiTS+phantom_249",
                                                                               "3d_fullres_LiTS+phantom_257",
                                                                               "3d_fullres_LiTS+phantom_258",
                                                                               "3d_fullres_LiTS+phantom_259"])) &
                                      (lesion_seg_summary_rep["reco+kernel"] == "HIR")],
              x="CTDIvol", y="F1-score",
             color="network", facet_row="ct", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             category_orders={"network": ["<b>3D improved", "<b>3D", "<b>3Dres improved", "<b>3Dres"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)


fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_points_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & 
                                      (lesion_seg_summary_rep["network"].isin(["<b>3Dres", "<b>3Dres improved"])) &
                                      (lesion_seg_summary_rep["reco+kernel"] == "HIR")],
              x="CTDIvol", y="F1-score",
             color="network", facet_row="ct", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"network": ["<b>3Dres improved", "<b>3Dres"]},)    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=600
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_ge_HIR_3D+3Dres_baseline_vs_improved_points_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:



df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean[df_lesion_meta_recall_rep_lesion_prop_mean["ct"] == "<b>Vendor 1"] 
df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean_filtered[df_lesion_meta_recall_rep_lesion_prop_mean_filtered["reco+kernel"].isin(["HIR"])]
df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean_filtered[df_lesion_meta_recall_rep_lesion_prop_mean_filtered["network"].isin(["<b>3Dres", "<b>3D", "<b>3Dres improved", "<b>3D improved"])]
df_lesion_meta_recall_rep_lesion_prop_mean_filtered["CTDIvol"] = df_lesion_meta_recall_rep_lesion_prop_mean_filtered["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_rep_lesion_prop_mean_filtered, x="CTDIvol", y="recall_min", color="network",
         facet_row="network_type", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D improved", "<b>3D", "<b>3Dres improved",  "<b>3Dres"],
                          "network_type": ["<b>3D", "<b>3Dres"]})

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=18, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_lesion_groups_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:

lesion_seg_baseline = lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 1") & (lesion_seg_summary["network"].isin(["<b>3D", "<b>3Dres"]))].copy()
lesion_seg_improved = lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 1") & (lesion_seg_summary["network"].isin(["<b>3D improved", "<b>3Dres improved"]))].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["improvement_rel"] = lesion_seg_improved["improvement"] / lesion_seg_baseline["F1-score"].values

fig = px.bar(lesion_seg_improved,
                x="CTDIvol", y="improvement_rel",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0, 1.4], barmode="group", template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score rel improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_rel_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:

lesion_seg_baseline = lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep_mean["network"].isin(["<b>3D", "<b>3Dres"]))].copy()
lesion_seg_improved = lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep_mean["network"].isin(["<b>3D improved", "<b>3Dres improved"]))].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["CTDIvol"] = lesion_seg_improved["CTDIvol"].astype(float).round(1).astype(str)

fig = px.bar(lesion_seg_improved,
                x="CTDIvol", y="improvement",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0, 1.4], barmode="group", template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score abs improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
lesion_seg_baseline = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"])) & (lesion_seg_summary_rep["reco+kernel"] == "HIR") ].copy()
lesion_seg_improved = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep["network"].isin(["<b>3D improved", "<b>3Dres improved"])) & (lesion_seg_summary_rep["reco+kernel"] == "HIR") ].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["improvement_rel"] = lesion_seg_improved["improvement"] / lesion_seg_baseline["F1-score"].values

fig = px.strip(lesion_seg_improved,
                x="CTDIvol", y="improvement_rel",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0,1.4], template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2")]
lesion_seg_summary_rep_plot =  lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["network"].isin(["<b>3D improved", "<b>3Dres improved"])]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="reco+kernel", facet_row="ct", facet_col="network", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D improved", "<b>3Dres improved"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_ge_improved_points_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:

fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2") & 
                                      (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3D improved", "<b>3Dres", "<b>3Dres improved"])) &
                                      (lesion_seg_summary_rep["reco+kernel"] == "HIR")],
              x="CTDIvol", y="F1-score",
             color="network", facet_row="ct", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"network": ["<b>3D improved", "<b>3D", "<b>3Dres improved", "<b>3Dres"]}, facet_col="network_type")    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.85, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)


fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_gr_HIR_3D+3Dres_baseline_vs_improved_points_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:

lesion_seg_baseline = lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 2") & (lesion_seg_summary["network"].isin(["<b>3D", "<b>3Dres"]))].copy()
lesion_seg_improved = lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 2") & (lesion_seg_summary["network"].isin(["<b>3D improved", "<b>3Dres improved"]))].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["improvement_rel"] = lesion_seg_improved["improvement"] / lesion_seg_baseline["F1-score"].values

fig = px.bar(lesion_seg_improved,
                x="CTDIvol", y="improvement_rel",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0, 1.4], barmode="group", template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score rel improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_ge_HIR_3D+3Dres_baseline_vs_improved_rel_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
lesion_seg_baseline = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"])) & (lesion_seg_summary_rep["reco+kernel"] == "HIR") ].copy()
lesion_seg_improved = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep["network"].isin(["<b>3D improved", "<b>3Dres improved"])) & (lesion_seg_summary_rep["reco+kernel"] == "HIR") ].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["improvement_rel"] = lesion_seg_improved["improvement"] / lesion_seg_baseline["F1-score"].values

fig = px.strip(lesion_seg_improved,
                x="CTDIvol", y="improvement_rel",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0,1.4], template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_ge_HIR_3D+3Dres_baseline_vs_improved_dicetreshold_{dice_detection_threshold}.png"))

In [ ]:

lesion_seg_baseline = lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep_mean["network"].isin(["<b>3D", "<b>3Dres"]))].copy()
lesion_seg_improved = lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep_mean["network"].isin(["<b>3D improved", "<b>3Dres improved"]))].copy()
lesion_seg_improved.loc[:,"improvement"] = lesion_seg_improved["F1-score"].values - lesion_seg_baseline["F1-score"].values
lesion_seg_improved["CTDIvol"] = lesion_seg_improved["CTDIvol"].astype(float).round(1).astype(str)

fig = px.bar(lesion_seg_improved,
                x="CTDIvol", y="improvement",
                color="reco+kernel", facet_row="ct", facet_col="network", 
                title="F1-score improvement for lesion segmentation", range_y=[0, 1.4], barmode="group", template="simple_white", 
                color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
                category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score abs improvement", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_HIR_3D+3Dres_baseline_vs_improved_dicetreshold_{dice_detection_threshold}.png"))


In [ ]:
df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean[df_lesion_meta_recall_rep_lesion_prop_mean["ct"] == "<b>Vendor 2"] 
df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean_filtered[df_lesion_meta_recall_rep_lesion_prop_mean_filtered["reco+kernel"].isin(["HIR"])]
df_lesion_meta_recall_rep_lesion_prop_mean_filtered = df_lesion_meta_recall_rep_lesion_prop_mean_filtered[df_lesion_meta_recall_rep_lesion_prop_mean_filtered["network"].isin(["<b>3Dres", "<b>3D", "<b>3Dres improved", "<b>3D improved"])]
df_lesion_meta_recall_rep_lesion_prop_mean_filtered["network_type"] = df_lesion_meta_recall_rep_lesion_prop_mean_filtered["network"].apply(lambda x: x.split(" ")[0])
df_lesion_meta_recall_rep_lesion_prop_mean_filtered["CTDIvol"] = df_lesion_meta_recall_rep_lesion_prop_mean_filtered["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_rep_lesion_prop_mean_filtered, x="CTDIvol", y="recall_min", color="network",
         facet_row="network_type", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D improved", "<b>3D", "<b>3Dres improved",  "<b>3Dres"],
                          "network_type": ["<b>3D", "<b>3Dres"]})

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=18, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_ge_HIR_3D+3Dres_baseline_vs_improved_lesion_groups_dicetreshold_{dice_detection_threshold}.png"))